In [15]:
from keras.models import Model
from keras.layers import Input
from keras.models import load_model
import string
from pyvi import ViTokenizer
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [16]:
model = load_model('NLP_LSTM_model_Encoder_Decoder.h5')

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 50)       243050      input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 20, 128), (N 91648       embedding[0][0]              

In [18]:
with open('word2index.pkl', 'rb') as f:
    word2index = pickle.load(f)

with open('index2word.pkl', 'rb') as f:
    index2word = pickle.load(f)

In [19]:
def text_process(mess):
    # chuyển về chữ thường
    mess = mess.lower()
    
    # xóa dấu câu
    mess = [char for char in mess if char not in string.punctuation]
    mess = ''.join(mess)
    
    # replace whitespace
    mess = mess.replace("   ", " ")
    mess = mess.replace("  ", " ")
    
    # Word Segmentation
    mess = ViTokenizer.tokenize(mess)
    
    return mess

In [20]:
MAX_LEN = 20

In [21]:
# Load model

latent_dim = 128

embed = model.layers[2] # Embedding

encoder_inputs = model.input[0]   # input_1
decoder_inputs = model.input[1]   # input_2

encoder_outputs, state_h, state_c = model.layers[3].output # lstm
encoder_states = [state_h, state_c]


decoder_lstm = model.layers[4] # lstm


dense = model.layers[5] # dense

In [22]:
# Để giải mã một câu:
# 1/ Encode câu đầu vào và truy xuất trạng thái ban đầu của encoder
# 2) Chạy một bước của decoder với trạng thái ban đầu và token "SOS" làm target. Ouput sẽ là từ tiếp theo.
# 3) Nối từ được dự đoán và lặp lại.

In [23]:
# Xác định model cho đầu vào - question. Trả về các final state vectơr của encoder LSTM 
encoder_model = Model([encoder_inputs], encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(embed(decoder_inputs), initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
output = dense(decoder_outputs) 
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [output] + decoder_states)

In [29]:
## Q-A

question_input = ""
while question_input != 'stop':
    question_input  = input("Q : ")
    
    if(question_input == "stop"): break
        
    if(question_input == ""): 
        print("A : Vui lòng nhập câu hỏi")
        print("===========================================================================================")
        question_input  = input("Q : ")
        
    question_input = text_process(question_input)
    
    question_inp = [question_input]
    
    question = []
    
    # chuyển câu trong question thành vector
    for x in question_inp:
        tmp = []
        for y in x.split():
            try:
                tmp.append(word2index[y])
            except:
                tmp.append(word2index['<OUT>'])
        question.append(tmp)
        
    # độ dài vector bằng MAX_LEN
    question = pad_sequences(question, MAX_LEN, padding='post')

    # chuyển câu hỏi cho encoder LSTM để nhận final encoder states của the encoder LSTM
    state = encoder_model.predict(question)
    
    # Tạo chuỗi mục tiêu trống có độ dài 1, tạo đầu vào là index của <SOS>
    empty_target_seq = np.zeros((1, 1))
    
    empty_target_seq[0, 0] = word2index['<SOS>']
    
    # điều kiện dừng decode
    run = True

    answer = ''

    while run:
        # Trả về prediction từ state của encoder và index từ trước đó
        prediction, prediction_h, prediction_c = decoder_model.predict([empty_target_seq] + state)
        
        # Từ prediction tạo ra tìm index với max probability
        token_word_index = np.argmax(prediction[0, -1, :])
        
        # Nối từ tương ứng với index vào word
        word = index2word[token_word_index] + ' '

        # Nếu word không phải EOS thì nối vào answer
        if(word != '<EOS> '):
            answer += word
            
        # Nếu word là EOS hoặc số từ vượt quá MAX_LEN thì dừng predict
        if(word == '<EOS> ' or len(answer.split()) > MAX_LEN):
            run = False

        # Khởi tạo lại empty_target_seq và đặt token_word_index thành đầu ra của current time step.
        # Sau đó nó sẽ chuyển làm đầu vào của next time step.
        empty_target_seq = np.zeros((1, 1))  
        empty_target_seq[0, 0] = token_word_index
        
        # Update states
        state = [prediction_h, prediction_c]
        
    print("A : ", answer)
    print("===========================================================================================")  

Q : mày thích đi xem phim hay là đi ăn?
A :  ok mai 9h đi 
Q : mày học ở đâu?
A :  quận 7 
Q : mày học ở đâu và làm gì ?
A :  ở tây_ninh của làng đại_học tôn đức thắng 
Q : mày học trường gì?
A :  em tích_lũy đủ 132 tín là tốt_nghiệp mình 
Q : stop
